In [5]:
import sys
import os
!ln -s /data /habitat-api/data
!ln -s /data ./data
sys.path.append('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [12]:
import habitat
from habitat.sims.habitat_simulator.actions import HabitatSimActions
import rospy
import rosgraph
from sensor_msgs.msg import Image, CameraInfo
from nav_msgs.msg import Path, Odometry
from geometry_msgs.msg import Pose, PoseStamped
from cv_bridge import CvBridge
import numpy as np
import keyboard
import argparse
import transformations as tf
from typing import Any
from gym import spaces
from habitat.utils.visualizations import maps
from skimage.io import imsave
from tqdm import tqdm
import h5py

In [4]:
rate = 20
D = [0, 0, 0, 0, 0]
K = [160, 0.0, 160.5, 0.0, 160, 120.5, 0.0, 0.0, 1.0]
R = [1, 0, 0, 0, 1, 0, 0, 0, 1]
P = [160, 0.0, 160.5, 0.0, 0.0, 160, 120.5, 0.0, 0.0, 0.0, 1.0, 0.0]
MAX_DEPTH = 100

def inverse_transform(x, y, start_x, start_y, start_angle):
    new_x = (x - start_x) * np.cos(start_angle) + (y - start_y) * np.sin(start_angle)
    new_y = -(x - start_x) * np.sin(start_angle) + (y - start_y) * np.cos(start_angle)
    return new_x, new_y

def get_local_pointcloud(rgb, depth, fov=90):
    fov = fov / (180 / np.pi)
    H, W, _ = rgb.shape
    idx_h = np.tile(np.arange(H), W).reshape((W, H)).T.astype(np.float32) - 120
    idx_w = np.tile(np.arange(W), H).reshape((H, W)).astype(np.float32) - 160
    print(W, (W / 2 * np.tan(fov / 2)))
    idx_h /= (W / 2 * np.tan(fov / 2))
    idx_w /= (W / 2 * np.tan(fov / 2))
    points = np.array([np.ones((H, W)), -idx_w, -idx_h])
    points = np.transpose(points, [1, 2, 0])
    points_dist = np.sqrt(np.sum(points ** 2, axis=2))
    #points = points / points_dist[:, :, np.newaxis] * depth * 10.0
    points = points * depth * MAX_DEPTH
    points = np.array([points[:, :, 0].ravel(), points[:, :, 1].ravel(), points[:, :, 2].ravel()]).T
    return points

# Define the sensor and register it with habitat
# For the sensor, we will register it with a custom name
@habitat.registry.register_sensor(name="position_sensor")
class AgentPositionSensor(habitat.Sensor):
    def __init__(self, sim, config, **kwargs: Any):
        super().__init__(config=config)

        self._sim = sim

    # Defines the name of the sensor in the sensor suite dictionary
    def _get_uuid(self, *args: Any, **kwargs: Any):
        return "agent_position"

    # Defines the type of the sensor
    def _get_sensor_type(self, *args: Any, **kwargs: Any):
        return habitat.SensorTypes.POSITION

    # Defines the size and range of the observations of the sensor
    def _get_observation_space(self, *args: Any, **kwargs: Any):
        return spaces.Box(
            low=np.finfo(np.float32).min,
            high=np.finfo(np.float32).max,
            shape=(3,),
            dtype=np.float32,
        )

    # This is called whenver reset is called or an action is taken
    def get_observation(
        self, observations, *args: Any, episode, **kwargs: Any
    ):
        sensor_states = self._sim.get_agent_state().sensor_states
        return (sensor_states['rgb'].position, sensor_states['rgb'].rotation)


class KeyboardAgent(habitat.Agent):
    def __init__(self, 
                 save_observations=True,
                 rgb_topic='/habitat/rgb/image',
                 depth_topic='/habitat/depth/image',
                 camera_info_topic='/habitat/rgb/camera_info',
                 path_topic='/true_path',
                 odometry_topic='/true_path',
                 publish_odom=True):
        rospy.init_node('agent')
        self.save_observations = save_observations
        self.image_publisher = rospy.Publisher(rgb_topic, Image, latch=True, queue_size=100)
        self.depth_publisher = rospy.Publisher(depth_topic, Image, latch=True, queue_size=100)
        self.camera_info_publisher = rospy.Publisher(camera_info_topic, CameraInfo, latch=True, queue_size=100)
        self.true_path_publisher = rospy.Publisher(path_topic, Path, queue_size=100)
        self.publish_odom = publish_odom
        if self.publish_odom:
            self.odom_publisher = rospy.Publisher(odometry_topic, Odometry, latch=True, queue_size=100)
        self.image = Image()
        self.image.height = 240
        self.image.width = 320
        self.image.encoding = 'rgb8'
        self.image.is_bigendian = False
        self.depth = Image()
        self.depth.height = 240
        self.depth.width = 320
        self.depth.is_bigendian = True
        self.depth.encoding = 'mono8'
        self.camera_info = CameraInfo(width=320, height=240, D=D, K=K, R=R, P=P) 
        self.cvbridge = CvBridge()
        self.trajectory = []
        self.map_path_subscriber = rospy.Subscriber('mapPath', Path, self.mappath_callback)
        self.slam_start_time = -1000
        self.slam_update_time = -1000
        self.is_started = False
        self.points = []
        self.positions = []
        self.rotations = []
        self.rgbs = []
        self.depths = []
        self.actions = []
        self.timestamps = []

    def mappath_callback(self, data):
        mappath_pose = data.poses[-1].pose
        x, y, z = mappath_pose.position.x, mappath_pose.position.y, mappath_pose.position.z
        xx, yy, zz, w = mappath_pose.orientation.x, mappath_pose.orientation.y, mappath_pose.orientation.z, mappath_pose.orientation.w
        cur_time = rospy.Time.now().secs + rospy.Time.now().nsecs * 1e-9
        eps = 1e-5
        if cur_time - self.slam_update_time > 30:
            self.slam_start_time = cur_time
            start_orientation = self.trajectory[-1].pose.orientation
            start_position = self.trajectory[-1].pose.position
            x_angle, z_angle, y_angle = tf.euler_from_quaternion([start_orientation.x, start_orientation.y, start_orientation.z, start_orientation.w])
            self.slam_start_angle = z_angle
            self.slam_start_x = start_position.x
            self.slam_start_y = start_position.y
            self.slam_start_z = start_position.z
            self.trajectory = []
        self.slam_update_time = cur_time

    def reset(self):
        pass

    def get_actions_from_keyboard(self):
        keyboard_commands = []
        if keyboard.is_pressed('left'):
            keyboard_commands.append(HabitatSimActions.TURN_LEFT)
        if keyboard.is_pressed('right'):
            keyboard_commands.append(HabitatSimActions.TURN_RIGHT)
        if keyboard.is_pressed('up'):
            keyboard_commands.append(HabitatSimActions.MOVE_FORWARD)
        return keyboard_commands

    def publish_rgb(self, image):
        start_time = rospy.Time.now()
        self.image = self.cvbridge.cv2_to_imgmsg(image)
        self.image.encoding = 'rgb8'
        self.image.header.stamp = start_time
        self.image.header.frame_id = 'camera_link'
        self.image_publisher.publish(self.image)

    def publish_depth(self, depth):
        start_time = rospy.Time.now()
        self.depth = self.cvbridge.cv2_to_imgmsg(depth * MAX_DEPTH)
        self.depth.header.stamp = start_time
        self.depth.header.frame_id = 'base_scan'
        self.depth_publisher.publish(self.depth)

    def publish_camera_info(self):
        start_time = rospy.Time.now()
        self.camera_info.header.stamp = start_time
        self.camera_info_publisher.publish(self.camera_info)

    def publish_true_path(self, pose, publish_odom):
        # count current coordinates and direction in global coords
        start_time = rospy.Time.now()
        position, rotation = pose
        y, z, x = position
        cur_orientation = rotation
        cur_euler_angles = tf.euler_from_quaternion([cur_orientation.w, cur_orientation.x, cur_orientation.z, cur_orientation.y])
        cur_x_angle, cur_y_angle, cur_z_angle = cur_euler_angles
        cur_z_angle += np.pi
        print('Source position:', y, z, x)
        print('Source quat:', cur_orientation.x, cur_orientation.y, cur_orientation.z, cur_orientation.w)
        print('Euler angles:', cur_x_angle, cur_y_angle, cur_z_angle)
        #print('After tf:', tf.quaternion_from_euler(cur_x_angle, cur_y_angle, cur_z_angle))
        if self.publish_odom:
            self.slam_update_time = start_time.secs + 1e-9 * start_time.nsecs
            if not self.is_started:
                self.is_started = True
                self.slam_start_angle = cur_z_angle
                print("SLAM START ANGLE:", self.slam_start_angle)
                self.slam_start_x = x
                self.slam_start_y = y
                self.slam_start_z = z
        # if SLAM is running, transform global coords to RViz coords
        if self.publish_odom or (start_time.secs + start_time.nsecs * 1e-9) - self.slam_update_time < 30:
            rviz_x, rviz_y = inverse_transform(x, y, self.slam_start_x, self.slam_start_y, self.slam_start_angle)
            rviz_z = z - self.slam_start_z
            cur_quaternion = tf.quaternion_from_euler(0, 0, cur_z_angle - self.slam_start_angle)
            print('Rotated quat:', cur_quaternion)
            cur_orientation.w = cur_quaternion[0]
            cur_orientation.x = cur_quaternion[1]
            cur_orientation.y = cur_quaternion[2]
            cur_orientation.z = cur_quaternion[3]
            x, y, z = rviz_x, rviz_y, rviz_z
        self.positions.append(np.array([x, y, z]))
        self.rotations.append(tf.quaternion_matrix(cur_quaternion))
        # add current point to path
        cur_pose = PoseStamped()
        cur_pose.header.stamp = start_time
        cur_pose.pose.position.x = x
        cur_pose.pose.position.y = y
        cur_pose.pose.position.z = z
        cur_pose.pose.orientation = cur_orientation
        self.trajectory.append(cur_pose)
        # publish the path
        true_path = Path()
        true_path.header.stamp = start_time
        true_path.header.frame_id = 'map'
        true_path.poses = self.trajectory
        self.true_path_publisher.publish(true_path)
        # publish odometry
        if self.publish_odom:
            odom = Odometry()
            odom.header.stamp = start_time
            odom.header.frame_id = 'odom'
            odom.child_frame_id = 'base_link'
            odom.pose.pose = cur_pose.pose
            self.odom_publisher.publish(odom)

    def act(self, observations):
        # publish all observations to ROS
        start_time = rospy.Time.now()
        pcd = get_local_pointcloud(observations['rgb'], observations['depth'])
        print(pcd.shape)
        if self.save_observations:
            self.points.append(pcd)
            self.rgbs.append(observations['rgb'].reshape((240 * 320, 3)))
            self.depths.append(observations['depth'])
            cur_time = rospy.Time.now()
            self.timestamps.append(cur_time.secs + 1e-9 * cur_time.nsecs)
        #self.positions.append(observations['agent_position'][0])
        #quaternion = observations['agent_position'][1]
        #rotation_matrix = [quaternion.w, quaternion.x, quaternion.y, quaternion.z]
        #self.rotations.append(rotation_matrix)
        self.publish_rgb(observations['rgb'])
        self.publish_depth(observations['depth'])
        self.publish_camera_info()
        self.publish_true_path(observations['agent_position'], self.publish_odom)
        # receive command from keyboard and move
        actions = self.get_actions_from_keyboard()
        start_time_seconds = start_time.secs + start_time.nsecs * 1e-9
        cur_time = rospy.Time.now()
        cur_time_seconds = cur_time.secs + cur_time.nsecs * 1e-9
        # make act time (1/rate) seconds
        time_left = cur_time_seconds - start_time_seconds
        if len(actions) > 0:
            rospy.sleep(1. / (rate * len(actions)) - time_left)
            action = np.random.choice(actions)
        else:
            rospy.sleep(1. / rate)
            action = HabitatSimActions.STOP
        self.actions.append(str(action))
        return action


def build_pointcloud(sim, discretization=0.05, grid_size=500, num_samples=20000):
    range_x = (np.inf, -np.inf)
    range_y = (np.inf, -np.inf)
    range_z = (np.inf, -np.inf)
    pointcloud = set()
    for i in range(num_samples):
        point = sim.sample_navigable_point()
        x, z, y = point
        z = np.random.random() * 3
        range_x = (min(range_x[0], x), max(range_x[1], x))
        range_y = (min(range_y[0], y), max(range_y[1], y))
        range_z = (min(range_z[0], z), max(range_z[1], z))
    for x in tqdm(np.linspace(range_x[0], range_x[1], grid_size)):
        for y in np.linspace(range_y[0], range_y[1], grid_size):
            for z in np.linspace(range_z[0], range_z[1], 100):
                closest_obstacle_point = sim._sim.pathfinder.closest_obstacle_surface_point(np.array([x, z, y])).hit_pos
                x_, z_, y_ = closest_obstacle_point
                x_ = np.round(x_ / discretization) * discretization
                y_ = np.round(y_ / discretization) * discretization
                z_ = np.round(z_ / discretization) * discretization
                pointcloud.add((x_, y_, z_))
    return np.array(list(pointcloud))

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--task-config", type=str, default="configs/tasks/pointnav.yaml")
parser.add_argument("--publish-odom", type=bool, default=True)
parser.add_argument("--create-map", type=bool, default=False)
parser.add_argument("--save-observations", type=bool, default=False)
parser.add_argument("--preset-trajectory", type=bool, default=False)
args = parser.parse_args("")
# Now define the config for the sensor
config = habitat.get_config(config_paths="../habitat-api/configs/tasks/pointnav.yaml")
config.defrost()
config.TASK.MEASUREMENTS.append("TOP_DOWN_MAP")
config.TASK.SENSORS.append("HEADING_SENSOR")
config.SIMULATOR.SCENE = 'data/scene_datasets/gibson/Aldrich.glb'
config.freeze()
max_depth = config.SIMULATOR.DEPTH_SENSOR.MAX_DEPTH
print(args.create_map)

False


In [11]:
agent = KeyboardAgent(args.save_observations)
env = habitat.Env(config=config)

2020-02-19 12:02:35,414 Initializing dataset PointNav-v1
2020-02-19 12:02:35,757 initializing sim Sim-v0
I0219 12:02:37.179933 441 simulator.py:142] Loaded navmesh data/scene_datasets/habitat-test-scenes/skokloster-castle.navmesh
2020-02-19 12:02:37,191 Initializing task Nav-v0


In [13]:
print('create map')
top_down_map = maps.get_topdown_map(env.sim, map_resolution=(5000, 5000))
print(top_down_map.min(), top_down_map.mean(), top_down_map.max())
recolor_map = np.array([[0, 0, 0], [128, 128, 128], [255, 255, 255]], dtype=np.uint8)
range_x = np.where(np.any(top_down_map, axis=1))[0]
range_y = np.where(np.any(top_down_map, axis=0))[0]
padding = int(np.ceil(top_down_map.shape[0] / 125))
range_x = (
    max(range_x[0] - padding, 0),
    min(range_x[-1] + padding + 1, top_down_map.shape[0]),
)
range_y = (
    max(range_y[0] - padding, 0),
    min(range_y[-1] + padding + 1, top_down_map.shape[1]),
)
top_down_map = top_down_map[
    range_x[0] : range_x[1], range_y[0] : range_y[1]
]
top_down_map = recolor_map[top_down_map]
imsave('top_down_map.png', top_down_map)
observations = env.reset()

create map
0 0.00988748 2


In [16]:
action = agent.act(observations)

ROSInitException: time is not initialized. Have you called init_node()?